In [ ]:
from hashlib import sha1
import hmac
import binascii
def getUrl(request):
    devId = 3002228
    key = '5bef2099-6493-48e4-b344-e37e99530ce6'
    request = request + ('&' if ('?' in request) else '?')
    raw = request+'devid={0}'.format(devId)
    #print(raw.encode())
    key_bin = key.encode()
    raw_bin = raw.encode() 
    hashed = hmac.new(key_bin, raw_bin, sha1)
    signature = hashed.hexdigest()
    return 'http://timetableapi.ptv.vic.gov.au'+raw+'&signature={1}'.format(devId, signature)
#print(getUrl('/v3/route_types'))


import requests
import json
def download_get(url_name,file_path):
    url_str = getUrl(url_name)
    print(url_str)
    response = requests.get(url= url_str)
    print(response.status_code)
    print(response.text)
    with open(file_path,'w') as f:
        f.write(response.text)
def download_get_to_dict(url_name):
    url_str = getUrl(url_name)
    print(url_str)
    response = requests.get(url= url_str)
    print(response.status_code)
    print(response.text)
    return json.loads(response.text)

#download_get('/v3/route_types','/Users/chunhua/tmp_share/heyonglin/route_types.json')
#download_get('/v3/routes','/Users/heyonglin/routes.json')

routes_json_file = '../data/routes.json'
# routes_json_file = '../data/routes.json'
with open(routes_json_file) as f:
    route_dict = json.load(f)
    # print(route_dict.keys())
    stops_files = [open(f'../data/stops_file_{i}.json','a')  for i in range(5)]
    for stop_file in stops_files:
        stop_file.write('[\n')
    is_first_line = [True,True,True,True,True]
    for route_item in route_dict['routes']:
        route_id = route_item['route_id']
        route_type = route_item['route_type']
        url_name = f'/v3/stops/route/{route_id}/route_type/{route_type}'
        res = download_get_to_dict(url_name)
        stops = res['stops']
        for stop in stops:
            if not is_first_line[route_type]:
                stops_files[route_type].write(',\n')
            json.dump(stop,stops_files[route_type])
            is_first_line[route_type] = False
    
    for stop_file in stops_files:
        stop_file.write('\n]')
        stop_file.close()    

In [9]:
def to_lower(text):
    text = text.lower()
    return text

In [11]:
import pandas as pd
train_stops_file = pd.read_json('../data/stops_file_0.json')
train_stops_file = train_stops_file[['stop_suburb','stop_latitude','stop_longitude','stop_id','stop_name']]
train_stops_file['stop_suburb'] = train_stops_file['stop_suburb'].apply(to_lower)
train_stops_file

,stop_suburb,stop_latitude,stop_longitude,stop_id,stop_name
0,ashburton,-37.868317,145.079666,1002,Alamein Station
1,ashburton,-37.861970,145.081345,1010,Ashburton Station
2,hawthorn east,-37.822400,145.045837,1012,Auburn Station
3,burnley,-37.827557,145.007553,1030,Burnley Station
4,glen iris,-37.851562,145.080500,1031,Burwood Station
...,...,...,...,...,...
365,melbourne city,-37.818310,144.966965,1071,Flinders Street Station
366,melbourne city,-37.809936,144.962600,1120,Melbourne Central Station
367,west melbourne,-37.806310,144.941513,1144,North Melbourne Station
368,melbourne city,-37.811054,144.972900,1155,Parliament Station


In [13]:
tram_stops_file = pd.read_json('../data/stops_file_1.json')
tram_stops_file = tram_stops_file[['stop_suburb','stop_latitude','stop_longitude','stop_id','stop_name']]
tram_stops_file['stop_suburb'] = tram_stops_file['stop_suburb'].apply(to_lower)
tram_stops_file

,stop_suburb,stop_latitude,stop_longitude,stop_id,stop_name
0,brunswick east,-37.769787,144.971954,2227,Albert St/Lygon St #123
1,brunswick east,-37.761890,144.974060,2240,Albion St/Holmes St #127
2,brunswick east,-37.762370,144.973267,2231,Albion St/Lygon St #127
3,southbank,-37.821842,144.969513,2203,Arts Precinct/St Kilda Rd #14
4,southbank,-37.823814,144.967255,2266,Arts Precinct/Sturt St #17
...,...,...,...,...,...
1518,melbourne city,-37.835762,144.975342,2612,Toorak Rd/St Kilda Rd #22
1519,glen iris,-37.858078,145.041046,2622,Tooronga Rd/High St #48
1520,brunswick east,-37.768204,144.972229,2228,Victoria St/Lygon St #124
1521,brunswick east,-37.775670,144.970978,2225,Weston St/Lygon St #121


In [14]:
bus_stops_file = pd.read_json('../data/stops_file_2.json')
bus_stops_file = bus_stops_file[['stop_suburb','stop_latitude','stop_longitude','stop_id','stop_name']]
bus_stops_file['stop_suburb'] = bus_stops_file['stop_suburb'].apply(to_lower)
bus_stops_file

,stop_suburb,stop_latitude,stop_longitude,stop_id,stop_name
0,avondale heights,-37.762234,144.859680,23570,Avondale Heights Reserve/Doyle St
1,maribyrnong,-37.769444,144.872772,31671,Canning Reserve/Canning St
2,avondale heights,-37.769722,144.871109,31670,Canning Reserve/Canning St
3,avondale heights,-37.769413,144.857254,25271,Canning St/Herbert St
4,maribyrnong,-37.769013,144.877823,11239,Central Park Ave/Cordite Ave
...,...,...,...,...,...
29805,hampton park,-38.028156,145.265900,30546,Village Dr/Pound Rd
29806,hampton park,-38.024235,145.269760,20093,Willow Dr/Hallam Rd
29807,wangaratta,-36.353348,146.324753,4532,King George Gardens/Rowan St
29808,glenrowan,-36.463898,146.221268,23857,Ned Kelly Statue/Kate St


In [15]:
vline_stops_file = pd.read_json('../data/stops_file_3.json')
vline_stops_file = vline_stops_file[['stop_suburb','stop_latitude','stop_longitude','stop_id','stop_name']]
vline_stops_file['stop_suburb'] = vline_stops_file['stop_suburb'].apply(to_lower)
vline_stops_file

,stop_suburb,stop_latitude,stop_longitude,stop_id,stop_name
0,ararat,-37.282440,142.936900,1501,Ararat Railway Station
1,ararat,-37.283640,142.931992,26759,Ararat Town Hall/Barkly St
2,macarthur,-38.033012,141.999741,21122,Eversley St/Hamilton - Port Fairy Rd
3,hamilton,-37.746600,142.028275,4125,Hamilton Station/Station St
4,dunkeld,-37.649464,142.342560,4130,Information Centre/Glenelg Hwy
...,...,...,...,...,...
1434,birregurra,-38.336193,143.783432,4151,Austin St/Warncoort-Birregurra Rd
1435,colac,-38.343376,143.586655,1517,Colac Railway Station
1436,waurn ponds,-38.198536,144.300140,15335,Deakin University/Alfred Deakin Dr
1437,geelong,-38.144240,144.355000,1527,Geelong Railway Station
